# Day 12 - Hill Climbing Algorithm

## Data

In [1]:
example_data = """
Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi
""".strip()

print(example_data)

Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi


In [2]:
import aocd
raw_data = aocd.get_data(year=2022, day=12)
print(raw_data)

abcccccccaaaaaccccaaaaaaaccccccccccccccccccccccccccccccccccccaaaaa
abaacccaaaaaaccccccaaaaaaaaaaaaaccccccccccccccccccccccccccccaaaaaa
abaacccaaaaaaaccccaaaaaaaaaaaaaacccccccccccccaacccccccccccccaaaaaa
abaacccccaaaaaacaaaaaaaaaaaaaaaacccccccccccccaacccccccccccccacacaa
abaccccccaaccaacaaaaaaaaaacccaacccccccccccccaaacccccccccccccccccaa
abcccccccaaaacccaaaaaaaaacccccccccccccaaacccaaacccccccccccccccccaa
abccccccccaaaccccccccaaaacccccccccccccaaaaacaaaccacacccccccccccccc
abccccccccaaacaaacccccaaacccccccccccccaaaaaaajjjjjkkkcccccaacccccc
abcccccaaaaaaaaaacccccaaccccccccccciiiiiijjjjjjjjjkkkcaaaaaacccccc
abcccccaaaaaaaaacccccccccccccccccciiiiiiijjjjjjjrrkkkkaaaaaaaacccc
abcccccccaaaaaccccccccccccccccccciiiiiiiijjjjrrrrrppkkkaaaaaaacccc
abcccaaccaaaaaacccccccccccaacaaciiiiqqqqqrrrrrrrrpppkkkaaaaaaacccc
abccaaaaaaaaaaaaccccacccccaaaaaciiiqqqqqqrrrrrruuppppkkaaaaacccccc
abcccaaaaaaacaaaacaaacccccaaaaaahiiqqqqtttrrruuuuupppkkaaaaacccccc
abcaaaaaaaccccaaaaaaacccccaaaaaahhqqqtttttuuuuuuuuuppkkkccaacc

In [3]:
from dataclasses import dataclass

directions = [(-1, 0), (0, -1), (1, 0), (0, 1)]

@dataclass
class Map(object):
    elevations: list[list[int]]
    start: tuple[int, int]
    end: tuple[int, int]
        
    def __post_init__(self):
        self.height = len(self.elevations)
        self.width = len(self.elevations[0]) 
        
    def __getitem__(self, key):
        x, y = key
        return self.elevations[y][x]
        
    def find_neighbors(self, x, y):
        for dx, dy in directions:
            nx = dx + x
            ny = dy + y
            
            if nx >= 0 and ny >= 0:
                if nx < self.width and ny < self.height:
                    yield (nx, ny)

## Parsing

In [4]:
def parse(data, debug = False):
    """Parse the input data, returning a Map object."""
    elevations = []
    
    for row, line in enumerate(data.split('\n')):
        section = []
        
        for col, elevation in enumerate(line):
            
            if ord(elevation) >= ord('a'):
                section.append(ord(elevation) - ord('a'))
            else:                
                if elevation == 'S':
                    start = (col, row)
                    section.append(0)
                else:
                    end = (col, row)
                    section.append(25)
                            
        elevations.append(section)
    
    return Map(elevations, start, end)
    
example_map = parse(example_data)
real_map = parse(raw_data)

print(example_map)

Map(elevations=[[0, 0, 1, 16, 15, 14, 13, 12], [0, 1, 2, 17, 24, 23, 23, 11], [0, 2, 2, 18, 25, 25, 23, 10], [0, 2, 2, 19, 20, 21, 22, 9], [0, 1, 3, 4, 5, 6, 7, 8]], start=(0, 0), end=(5, 2))


## Part 1

In [30]:
def find_paths(map, x, y, visited={}, distance=0, debug = False, inverse = False):
    """
    Finds the shortest distance to all points in the provided map.
    If inverse is true, we are traverse teh paths from highest elevation to lowest.
    """
    if (x, y) in visited and visited[(x, y)] <= distance:
        return
    if debug:
        print(f"Visiting {x}, {y} with distance {distance}")
    
    visited[(x, y)] = distance

    for nx, ny in map.find_neighbors(x, y):
        slope = map[(nx, ny)] - map[(x,y)]
        if inverse:
            slope = -slope
        
        if slope <= 1:
            find_paths(map, nx, ny, visited, distance+1, debug, inverse)

            
def calc_shortest_path(map):
    distances = {}
    find_paths(map, map.start[0], map.start[1], distances)
    return distances[map.end]

assert calc_shortest_path(example_map) == 31
calc_shortest_path(real_map)

339

## Part 2

In [33]:
def calc_hiking_trail(map):
    distances = {}
    find_paths(map, map.end[0], map.end[1], distances, inverse=True)

    down_hill_distances = []
    
    for x in range(map.width):
        for y in range(map.height):
            if map[x, y] == 0 and (x, y) in distances:
                down_hill_distances.append(distances[x, y])
    
    return min(down_hill_distances)

calc_hiking_trail(example_map)# == 31
calc_hiking_trail(real_map)

332